In [ ]:
# Google Colab Setup
# Install required packages
%pip install flask==2.3.3

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [1]:
# Agent Base Class
class Agent:
    def __init__(self, name):
        self.name = name

    def respond(self, input_text):
        raise NotImplementedError("Each agent must implement respond()")

In [2]:
# Claim Extractor Agent
class ClaimExtractor(Agent):
    def __init__(self):
        super().__init__("Claim Extractor")

    def respond(self, text):
        sentences = text.split(".")
        claims = [s.strip() for s in sentences if len(s.strip()) > 10]
        return claims

In [3]:
# Fact Checker Agent
class FactChecker(Agent):
    def __init__(self):
        super().__init__("Fact Checker")
        # Known facts for demo; in a real system, this could be expanded or use an API
        self.known_facts = {
            "the sky is blue": True,
            "cats are mammals": True,
            "the sky is not blue": False,
            "cats are not mammals": False
        }

    def respond(self, claims):
        verified = []
        for claim in claims:
            key = claim.lower().strip()
            if key in self.known_facts:
                status = "True" if self.known_facts[key] else "False"
            elif "always" in claim.lower() or "never" in claim.lower():
                status = "Suspicious"
            else:
                status = "Needs Verification"  # Avoid claiming "Likely True" for unknown claims to prevent misinformation
            verified.append((claim, status))
        return verified

In [9]:
# Contradiction Detector Agent
class ContradictionDetector(Agent):
    def __init__(self):
        super().__init__("Contradiction Detector")

    def respond(self, claims):
        contradictions = []
        for i in range(len(claims)):
            for j in range(i + 1, len(claims)):
                claim1 = claims[i].lower().strip()
                claim2 = claims[j].lower().strip()
                # Check if one is the negation of the other
                if claim1.replace("not ", "") == claim2 or claim2.replace("not ", "") == claim1:
                    contradictions.append((claims[i], claims[j]))
        return contradictions

In [5]:
# Moderator Agent
class Moderator(Agent):
    def __init__(self):
        super().__init__("Moderator")

    def respond(self, claims, facts, contradictions):
        report = {
            "total_claims": len(claims),
            "verified_claims": facts,
            "contradictions_found": contradictions,
            "consistency_score": max(0, 100 - (len(contradictions) * 20))
        }
        return report

In [6]:
# Debate Manager
class DebateManager:
    def __init__(self):
        self.extractor = ClaimExtractor()
        self.checker = FactChecker()
        self.detector = ContradictionDetector()
        self.moderator = Moderator()

    def run(self, text):
        claims = self.extractor.respond(text)
        facts = self.checker.respond(claims)
        contradictions = self.detector.respond(claims)
        final_report = self.moderator.respond(claims, facts, contradictions)
        return final_report

In [10]:
# Sample Text
text = """
The sky is blue.
The sky is not blue.
Cats are mammals.
Cats are not mammals.
"""

# Run Debate
manager = DebateManager()
report = manager.run(text)

print("\n--- FINAL CONSISTENCY REPORT ---")
for key, value in report.items():
    print(f"{key}: {value}")


--- FINAL CONSISTENCY REPORT ---
total_claims: 4
verified_claims: [('The sky is blue', 'True'), ('The sky is not blue', 'False'), ('Cats are mammals', 'True'), ('Cats are not mammals', 'False')]
contradictions_found: [('The sky is blue', 'The sky is not blue'), ('Cats are mammals', 'Cats are not mammals')]
consistency_score: 60
